In [1]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv('bbc.csv')

In [6]:
# construct the Word of Bag model
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer() 
X = vectorizer.fit_transform(df['text'])
word = vectorizer.get_feature_names()
word_bag = X.toarray()

In [10]:
len(word)

31352

In [11]:
len(word_bag)

2225

calculate the cosine similarity

In [12]:
def cos_sim(A, B):
    numerator = float(np.dot(np.mat(A), np.mat(B).T)) 
    denominator = np.linalg.norm(A) * np.linalg.norm(B) 
    if denominator  == 0:
        denominator  = 1
    cosn = numerator / denominator
    sim = 0.5 + 0.5 * cosn  # normalization to [0,1]
    sim = 1 - sim  # The smaller the value, the closer the distance
    return sim

work flow: 

（1）Randomly determine k initial points as the center of mass.

（2）Find the nearest center of mass for each point in the data set and assign it to the cluster corresponding to the center of mass

（3）Update the center of mass of each cluster to the average of all points in the cluster.

（4）Repeat steps (2) and (3) until the distribution result of the cluster does not change.

In [115]:
# based on the machine, we can determine to extract the number of points, in our example the number is 1000
import random
word_bag_temp = pd.DataFrame(word_bag)
randomlist = random.sample(range(0,word_bag.shape[0]),1000)
list = []
for i in randomlist:
    list.append(word_bag_temp.iloc[i])
test = pd.DataFrame(list).reset_index(drop = True)

In [45]:
#reduce the features to 100 in order accelerate the speed
from sklearn.decomposition import PCA
pca = PCA(n_components=100, random_state=0)
test = pca.fit_transform(test)

In [12]:
# Construct a random set of k centers of mass
def randCent(k):
    import random
    cent = []
    randomlist = random.sample(range(0,test.shape[0]),5)
    a = test.tolist()
    for i in randomlist:
        cent.append(a[i])
    return cent

In [58]:
def kMeans(k, distMeas = cos_sim, createCent= randCent):
    from math import inf
    m = test.shape[0]  # total number of vectors
    
    # The first column is the index of the center of mass, and the second column is the cosine similarity
    clusterAssment = np.mat(np.zeros((m, 2)))  
    centroids = createCent(k)  # Construct a random set of k centers of mass
    b = test.tolist() # Turn word_bag into list
    i = 0;
    while i<50:
        clusterChanged = False
        for i in range(m):  # Each point given in the total data
            minDist = inf  # INFINITE
            minIndex = -1
            for j in range(k):  # each center
                distJI = distMeas(centroids[j], b[i])  # calculate the cosine similarity
                if distJI < minDist:   
                    minDist = distJI
                    minIndex = j # record the corresponding point
            if clusterAssment[i, 0] != minIndex:
                clusterAssment[i,:] = minIndex, minDist 
            
        # print centroids
        for cent in range(k):
            ptsInClust = test[np.nonzero(clusterAssment[:,0].A == cent)[0]]
            centroids[cent] = np.mean(ptsInClust, axis=0)
    i = i+1
    return centroids, clusterAssment

In [59]:
centers,cluster = kMeans(5)

In [60]:
#reduce the features to 2D
from sklearn.decomposition import PCA
pca = PCA(n_components=2, random_state=0)
reduced_features = pca.fit_transform(test)

# reduce the cluster centers to 2D
reduced_cluster_centers = pca.transform(centers)